In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'/workspaces/Xray'

In [3]:
from src.utils.utils import read_yaml,create_dir
from src.constant.ymal_path import *
from src.logging.logger import logging
from src.exception.exception import CustomException
import sys



2024-05-04 16:11:47.071198: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 16:11:48.181915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import tensorflow
from tensorflow import keras

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class CustomModelConfig:
    dir:Path
    custom_Model_path:Path
    IMAGE_SIZE:list
    LEARNING_RATE:float
    CLASSES: int

In [6]:
class ConfigMager:
    def __init__(self,
                 config_file_path=config_file_path,
                 params_file_path=params_file_path) -> None:
        self.config=read_yaml(config_file_path)
        self.prams=read_yaml(params_file_path)

    def get_cusom_model_config(self):
        config=self.config.custom_model
        param=self.prams

        create_dir([config.dir])

        custom_model_config=CustomModelConfig(
            dir=config.dir,
            custom_Model_path=config.custom_Model_path,
            IMAGE_SIZE=param.IMAGE_SIZE,
            LEARNING_RATE=param.LEARNING_RATE,
            CLASSES=param.CLASSES
        )

        return custom_model_config

In [7]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Layer
from keras import layers,models

In [8]:
class CustomModel:
    def __init__(self,config=CustomModelConfig) -> None:
        self.config=config
    @staticmethod
    def save_model(path: Path, model: keras.Model):
        model.save(path)
    def initiate_custom_mode(self):
    
        try:
            logging.info('custom model creation started')
            
            model = models.Sequential([
            layers.Conv2D(128, (3, 3), activation='relu',input_shape=self.config.IMAGE_SIZE),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(32, (3, 3), activation='relu'),
             layers.Conv2D(16, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(self.config.CLASSES, activation='softmax')
        ])

            optimizer = keras.optimizers.SGD(learning_rate=self.config.LEARNING_RATE)
            loss = keras.losses.CategoricalCrossentropy()
            metrics = ["accuracy"]
            
            custom_model=model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
            
            model.summary()  # Print summary
            self.save_model(path=self.config.custom_Model_path,model=model)
            
            return model

            
        except Exception as e:
            logging.info('error occured ', e)
            raise CustomException(sys,e) 

In [9]:
try:
    config=ConfigMager()
    custom_model_config=config.get_cusom_model_config()
    custom_model=CustomModel(config=custom_model_config)
    custom_model.initiate_custom_mode()
except Exception as e:
            logging.info('error occured ', str(e))
            raise CustomException(sys,e) 

/home/codespace/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 50, 50, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 25, 25, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 740,658 (2.83 MB)

 Trainable params: 740,658 (2.83 MB)

 Non-trainable params: 0 (0.00 B)